## Amazon Bedrock Knowledge Bases - 다양한 데이터 소스를 사용한 엔드 투 엔드 예제 ##
이 노트북은 Amazon Bedrock Knowledge Bases를 사용하여 RAG 애플리케이션을 구축하고 다양한 데이터 소스(S3, Confluence, Sharepoint, Salesforce 및 Web)에서 문서를 색인에 수집하는 엔드 투 엔드 예제의 샘플 코드를 제공합니다. 최대 5개의 데이터 소스를 추가할 수 있습니다.

노트북 설명
질문이 수신될 때 조회할 수 있도록 문서(일반적으로 여러 데이터 소스에 저장됨)를 Amazon OpenSearch Service Serverless(AOSS)와 같은 벡터 데이터베이스인 지식 베이스로 수집하는 데이터 파이프라인입니다.

다양한 데이터 소스(S3, Confluence, Sharepoint, Salesforce 및 Web)를 연결하여 문서를 지식 베이스에 로드합니다.
수집 - 지식 베이스는 문서를 더 작은 청크로 분할하고(선택한 전략에 따라), 임베딩을 생성하여 관련 벡터 저장소에 저장합니다.
<!-- data_ingestion.png -->
<img src="./images/data_ingestion.png" width=50% height=20% />

단계:
다양한 데이터 소스(S3, Confluence, Sharepoint, Salesforce 및 Web)에서 데이터에 액세스하고 OSS에 임베딩을 쓰는 데 필요한 정책이 포함된 Knowledge Base 실행 역할을 생성합니다.
빈 OpenSearch 서버리스 인덱스를 생성합니다.
사전 요구 사항:
S3의 경우, s3 버킷 생성(존재하지 않는 경우) 및 데이터 업로드
다른 데이터 소스의 경우 - 해당 [AWS 문서 페이지](https://docs.aws.amazon.com/bedrock/latest/userguide/data-source-connectors.html)의 사전 요구 사항 참조
지식 베이스 생성
지식 베이스 내에 데이터 소스 생성
각 데이터 소스에 대해 KB API를 사용하여 수집 작업을 시작하면 데이터 소스에서 데이터를 읽고, 청크로 분할하고, Amazon Titan Embeddings 모델을 사용하여 청크를 임베딩으로 변환한 다음 이러한 임베딩을 AOSS에 저장합니다. 이 모든 것이 데이터 파이프라인을 구축, 배포 및 관리할 필요 없이 이루어집니다.
데이터가 Bedrock Knowledge Base에서 사용 가능하게 되면 Amazon Bedrock에서 제공하는 Knowledge Base API를 사용하여 질문 답변 애플리케이션을 구축할 수 있습니다.

<div class="alert alert-block alert-info">
<b>참고:</b> Amazon Bedrock Console에서 다음 모델들에 대한 접근 권한을 활성화했는지 확인하십시오:
amazon.nova-micro-v1:0
Anthropic Claude 3 Sonnet
amazon.titan-text-express-v1
anthropic.claude-3-haiku-20240307-v1:0
Titan Text Embeddings V2 
<br> ------------------------------------------------------------------------------------------------------------------------------------------------------- </br>
"모든 셀 실행" 옵션을 사용하는 대신 노트북의 셀을 하나씩 실행해 주시기 바랍니다.
</div>

## 설정
이 노트북의 나머지 부분을 실행하기 전에, (필요한 라이브러리가 설치되어 있는지 확인하고) Bedrock에 연결하기 위해 아래 셀들을 실행해야 합니다.

In [1]:
%pip install --upgrade pip --quiet
%pip install -r ../requirements.txt --no-deps --quiet
%pip install -r ../requirements.txt --upgrade --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.3.1 requires nvidia-ml-py3<8.0,>=7.352.0, which is not installed.
dash 2.18.1 requires dash-core-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-html-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-table==5.0.0, which is not installed.
aiobotocore 2.21.1 requires botocore<1.37.2,>=1.37.0, but you have botocore 1.39.17 which is incompatible.
autogluon-common 1.3.1 requires pandas<2.3.0,>=2.0.0, but you have pandas 2.3.1 which is incompatible.
autogluon-core 1.3.1 requires pandas<2.3.0,>=2.0.0, but you have pandas 2.3.1 which is incompatible.
autogluon-features 1.3.1 requires pandas<2.3.0,>=2.0.0, but you have pandas 2.3.1

In [2]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import os
import sys
import time
import boto3
import logging
import pprint
import json

# Set the path to import module
from pathlib import Path
current_path = Path().resolve()
current_path = current_path.parent
if str(current_path) not in sys.path:
    sys.path.append(str(current_path))

# Print sys.path to verify
print(sys.path)

from utils.knowledge_base import BedrockKnowledgeBase

['/opt/conda/lib/python312.zip', '/opt/conda/lib/python3.12', '/opt/conda/lib/python3.12/lib-dynload', '', '/opt/conda/lib/python3.12/site-packages', '/home/sagemaker-user/Advanced_RAG_Workshop_on_Amazon_Bedrock']


In [5]:
#Clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region =  session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime') 
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
region, account_id

('us-west-2', '461433424192')

In [6]:
import time

# Get the current timestamp
current_time = time.time()

# Format the timestamp as a string
timestamp_str = time.strftime("%Y%m%d%H%M%S", time.localtime(current_time))[-7:]
# Create the suffix using the timestamp
suffix = f"{timestamp_str}"

knowledge_base_name = f"bedrock-sample-knowledge-base-{suffix}"
knowledge_base_description = "Multi data source knowledge base."

bucket_name = f'{knowledge_base_name}-{account_id}'
intermediate_bucket_name = f'{knowledge_base_name}-intermediate-{account_id}'
foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"

In [7]:
print(boto3.__version__)

1.39.17


### 이제 지식 베이스에 여러 가지 다양한 데이터 소스(S3, Confluence, Sharepoint, Salesforce, Web Crawler)를 추가할 수 있습니다. 이 노트북에서는 여러 가지 다양한 데이터 소스를 사용하여 지식 베이스 생성을 테스트해보겠습니다.

각 데이터 소스마다 서로 다른 사전 요구사항이 있을 수 있으므로, 자세한 내용은 AWS 문서를 참조하시기 바랍니다.

In [8]:
# For this notebook, we'll create Knowledge Base with multiple data sources ( 1 S3 bucket, 1 confluence page, 1 Sharepoint site, 1 Salesforce site, 1 Web Crawler)

data_bucket_name = f'bedrock-kb-{suffix}-1' # replace it with your first bucket name.

## Below is a list of data sources including, 1 S3 buckets, 1 confluence, 1 Sharepoint, 1 Salesforce connectors
## Please uncomment the data sources that you want to add and update the placeholder values accordingly.

data_sources=[
                {"type": "S3", "bucket_name": data_bucket_name}, 
                
                # {"type": "CONFLUENCE", "hostUrl": "https://example.atlassian.net", "authType": "BASIC",
                #  "credentialsSecretArn": f"arn:aws::secretsmanager:{region_name}:secret:<<your_secret_name>>"},

                # {"type": "SHAREPOINT", "tenantId": "888d0b57-69f1-4fb8-957f-e1f0bedf64de", "domain": "yourdomain",
                #   "authType": "OAUTH2_CLIENT_CREDENTIALS",
                #  "credentialsSecretArn": f"arn:aws::secretsmanager:{region_name}:secret:<<your_secret_name>>",
                #  "siteUrls": ["https://yourdomain.sharepoint.com/sites/mysite"]
                # },

                # {"type": "SALESFORCE", "hostUrl": "https://company.salesforce.com/", "authType": "OAUTH2_CLIENT_CREDENTIALS",
                #  "credentialsSecretArn": f"arn:aws::secretsmanager:{region_name}:secret:<<your_secret_name>>"
                # },

                # {"type": "WEB", "seedUrls": [{ "url": "https://www.examplesite.com"}],
                #  "inclusionFilters": ["https://www\.examplesite\.com/.*\.html"],
                #  "exclusionFilters": ["https://www\.examplesite\.com/contact-us\.html"]
                # }
            ]
                
pp = pprint.PrettyPrinter(indent=2)

## Knowledge Base 생성하기

In [9]:
knowledge_base = BedrockKnowledgeBase(
    kb_name=f'{knowledge_base_name}',
    kb_description=knowledge_base_description,
    data_sources=data_sources,
    chunking_strategy = "FIXED_SIZE", 
    suffix = f'{suffix}-f'
)

Step 1 - Creating or retrieving S3 bucket(s) for Knowledge Base documents
['bedrock-kb-1063926-1']
buckets_to_check:  ['bedrock-kb-1063926-1']
Creating bucket bedrock-kb-1063926-1
Step 2 - Creating Knowledge Base Execution Role (AmazonBedrockExecutionRoleForKnowledgeBase_1063926-f) and Policies
Step 3a - Creating OSS encryption, network and data access policies
Step 3b - Creating OSS Collection (this step takes a couple of minutes to complete)
{ 'ResponseMetadata': { 'HTTPHeaders': { 'connection': 'keep-alive',
                                         'content-length': '320',
                                         'content-type': 'application/x-amz-json-1.0',
                                         'date': 'Thu, 31 Jul 2025 06:39:29 '
                                                 'GMT',
                                         'x-amzn-requestid': 'd895e693-8f6c-4351-af6f-2ebd03b51628'},
                        'HTTPStatusCode': 200,
                        'RequestId': 'd895e693-

[2025-07-31 06:44:01,355] p213 {base.py:258} INFO - PUT https://v048t3nygpr5y3xug945.us-west-2.aoss.amazonaws.com:443/bedrock-sample-rag-index-1063926-f [status:200 request:0.456s]



Creating index:
{ 'acknowledged': True,
  'index': 'bedrock-sample-rag-index-1063926-f',
  'shards_acknowledged': True}
Step 4 - Will create Lambda Function if chunking strategy selected as CUSTOM
Not creating lambda function as chunking strategy is FIXED_SIZE
Step 5 - Creating Knowledge Base
{ 'createdAt': datetime.datetime(2025, 7, 31, 6, 45, 1, 481741, tzinfo=tzlocal()),
  'description': 'Multi data source knowledge base.',
  'knowledgeBaseArn': 'arn:aws:bedrock:us-west-2:461433424192:knowledge-base/WX5DF4TRCJ',
  'knowledgeBaseConfiguration': { 'type': 'VECTOR',
                                  'vectorKnowledgeBaseConfiguration': { 'embeddingModelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-embed-text-v2:0'}},
  'knowledgeBaseId': 'WX5DF4TRCJ',
  'name': 'bedrock-sample-knowledge-base-1063926',
  'roleArn': 'arn:aws:iam::461433424192:role/AmazonBedrockExecutionRoleForKnowledgeBase_1063926-f',
  'status': 'CREATING',
  'storageConfiguration': { 'opensearchServer

### 지식 베이스에 수집할 데이터 다운로드
다음과 같은 데이터를 사용할 것입니다:
- 첫 번째 데이터 소스로 로컬 디렉토리에 저장된 sythetic_dataset 폴더 내 자료

#### S3 Bucket data source로 데이터 업로드

In [10]:
def upload_directory(path, bucket_name):
        for root,dirs,files in os.walk(path):
            for file in files:
                file_to_upload = os.path.join(root,file)
                print(f"uploading file {file_to_upload} to {bucket_name}")
                s3_client.upload_file(file_to_upload,bucket_name,file)

upload_directory("../synthetic_dataset", data_bucket_name)

uploading file ../synthetic_dataset/2024_population.pdf to bedrock-kb-1063926-1


### 수집 작업 시작
KB(Knowledge Base)와 데이터 소스가 생성되면, 각 데이터 소스에 대한 수집 작업을 시작할 수 있습니다.
수집 작업 중에 KB는 다음과 같은 작업을 수행합니다:
- 데이터 소스에서 문서를 가져옴
- 텍스트를 추출하기 위해 전처리
- 제공된 청크 크기를 기반으로 문서를 분할
- 각 청크의 임베딩을 생성
- 생성된 임베딩을 벡터 데이터베이스(이 경우 OSS)에 저장

주의: 현재는 한 번에 하나의 수집 작업만 시작할 수 있습니다.

In [11]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base.start_ingestion_job()

job 1 started successfully

{ 'dataSourceId': 'H3WUUYQWXY',
  'ingestionJobId': 'ULI3LGZSFC',
  'knowledgeBaseId': 'WX5DF4TRCJ',
  'startedAt': datetime.datetime(2025, 7, 31, 6, 52, 58, 763985, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 1,
                  'numberOfMetadataDocumentsModified': 0,
                  'numberOfMetadataDocumentsScanned': 0,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 1},
  'status': 'COMPLETE',
  'updatedAt': datetime.datetime(2025, 7, 31, 6, 53, 51, 340158, tzinfo=tzlocal())}
........................................

In [12]:
# keep the kb_id for invocation later in the invoke request
kb_id = knowledge_base.get_knowledge_base_id()
%store kb_id

'WX5DF4TRCJ'
Stored 'kb_id' (str)


### 2.2 Knowledge Base 테스트
이제 Knowledge Base를 사용할 수 있게 되었으므로 [retrieve](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve.html)와 [retrieve_and_generate](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html) 함수를 사용하여 테스트해볼 수 있습니다.

#### Retrieve and Generate API를 사용한 Knowledge Base 테스트
먼저 retrieve and generate API를 사용하여 Knowledge Base를 테스트해보겠습니다. 이 API를 통해 Bedrock은 지식 베이스에서 필요한 참조를 검색하고 Bedrock의 기반 모델을 사용하여 최종 답변을 생성하는 작업을 처리합니다.

In [13]:
query = "2024년 기준 시도별 노령화지수를 비교했을 때, 가장 높은 지역과 가장 낮은 지역의 차이가 발생하는 원인을 인구구조와 사회경제적 특성을 고려하여 분석해줘"
#query = "Provide a summary of consolidated statements of cash flows of Octank Financial for the fiscal years ended December 31, 2019?"

In [14]:
foundation_model = "anthropic.claude-3-haiku-20240307-v1:0"

response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region, foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

2024년 기준 시도별 노령화지수를 비교하면, 가장 높은 지역은 경북(215.0)이고 가장 낮은 지역은 수도권(167.2)으로 약 48 포인트의 차이가 발생합니다. 이러한 차이는 지역별 인구구조와 사회경제적 특성의 차이에서 기인합니다. 경북은 농촌 지역이 많아 젊은 층의 인구 유출이 지속되고 고령 인구 비율이 높은 편입니다. 반면 수도권은 상대적으로 경제 활동이 활발하고 일자리가 많아 젊은 층의 유입이 많아 노령화 수준이 낮습니다. 또한 수도권은 교육, 의료, 복지 등 사회 인프라가 잘 갖춰져 있어 고령자들의 삶의 질이 높은 편입니다.



보시다시피, retrieve 및 generate API를 사용하면 최종 응답을 직접 받을 수 있으며, 이 응답을 생성하는 데 사용된 다양한 소스는 확인할 수 없습니다. 이제 retrieve API를 사용하여 Knowledge Base에서 소스 정보를 검색해 보겠습니다.

#### Retrieve API를 사용한 Knowledge Base 테스트
추가적인 제어 계층이 필요한 경우, retrieve API를 사용하여 쿼리와 가장 잘 일치하는 청크들을 검색할 수 있습니다. 이 설정에서는:

- 원하는 결과 수를 구성할 수 있습니다
- 자체 애플리케이션 로직으로 최종 답변을 제어할 수 있습니다
- API는 다음을 제공합니다:
    - 일치하는 콘텐츠
    - S3 위치
    - 유사도 점수
    - 청크 메타데이터

In [16]:
response_ret = bedrock_agent_runtime_client.retrieve(
    knowledgeBaseId=kb_id, 
    nextToken='string',
    retrievalConfiguration={
        "vectorSearchConfiguration": {
            "numberOfResults":5,
        } 
    },
    retrievalQuery={
        #"text": "How many new positions were opened across Amazon's fulfillment and delivery network?"
        "text": "2024년 기준 시도별 노령화지수를 비교했을 때, 가장 높은 지역과 가장 낮은 지역의 차이가 발생하는 원인을 인구구조와 사회경제적 특성을 고려하여 분석해줘."
    }
)

def response_print(retrieve_resp):
#structure 'retrievalResults': list of contents. Each list has content, location, score, metadata
    for num,chunk in enumerate(response_ret['retrievalResults'],1):
        print(f'Chunk {num}: ',chunk['content']['text'],end='\n'*2)
        print(f'Chunk {num} Location: ',chunk['location'],end='\n'*2)
        print(f'Chunk {num} Score: ',chunk['score'],end='\n'*2)
        print(f'Chunk {num} Metadata: ',chunk['metadata'],end='\n'*2)

response_print(response_ret)

Chunk 1:  ) 순으로 크고, 세종(15.6), 경기(22.6)     순으로 작음      ❍전년 대비 노령화지수 증감은 경북(22.4)이 가장 높고, 세종(5.4)이 가장 낮음          < 표 8 > 시도별 인구의 연령 지표, 2023~2024년     (단위 : 세, 100명당)     시 도 중위연령 유소년부양비 노년부양비 노령화지수     2023년 2024년 증감 2023년 2024년 증감 2023년 2024년 증감 2023년 2024년 증감     전 국 45.7 46.2 0.6 15.4 14.9 -0.4 26.3 27.9 1.6 171.0 186.7 15.7 서 울 44.4 44.9 0.5 12.3

Chunk 1 Location:  {'s3Location': {'uri': 's3://bedrock-kb-1063926-1/2024_population.pdf'}, 'type': 'S3'}

Chunk 1 Score:  0.503256

Chunk 1 Metadata:  {'x-amz-bedrock-kb-source-uri': 's3://bedrock-kb-1063926-1/2024_population.pdf', 'x-amz-bedrock-kb-document-page-number': 29.0, 'x-amz-bedrock-kb-chunk-id': '1%3A0%3A-ChBX5gBlyhKdOCQkiWu', 'x-amz-bedrock-kb-data-source-id': 'H3WUUYQWXY'}

Chunk 2:  %) 지역에서 모두 감소      ❍ 동 지역과 면 지역의 중위연령 격차 12.0세(동 지역 45.1세, 면 지역 57.2세)      ❍ 노령화지수는 영남권(215.0)이 가장 높고, 수도권(167.2)이 가장 낮음      - 동 지역은 166.1, 면 지역은 524.8로 동 지역의 약 3.2배임      - 전년 대비 노령화지수 증가는 면 지역(45.7), 동 지역(14.5), 읍 지역(14.0) 순으로 큼          < 

## 리소스 정리
모든 리소스를 삭제하기 위해 아래 섹션의 주석을 해제하고 실행하시기 바랍니다.

In [17]:
# delete role and policies
print("===============================Deleting Knowledge Base and associated resources==============================\n")
# knowledge_base.delete_kb(delete_s3_bucket=True, delete_iam_roles_and_policies=True)

===============================Deleting Knowledge Base and associated resources==============================

